# 실종자의 사진을 수집하여 구글 드라이브에 저장하는 소스입니다


In [1]:
import re
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
from PIL import Image
import urllib.request
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
HTML_URI = 'drive/My Drive/development/html/'
IMG_URI = 'drive/My Drive/development/images/missing/'
IMG_URL = 'http://www.safe182.go.kr/blobImgView.do?msspsnIdntfccd='
IMG_PRM = '&rptDscd=2'

In [4]:
def identified():
  datetime.today()
  iid = datetime.today().strftime("%Y%m%d%H%M")

  return iid

In [5]:
def make_array(string, keyword): 
  arr = []
  arr = string.split(keyword)
  return arr

In [6]:
def make_date(string):
  numbers = re.findall('\d+', string)
  _year = numbers[0][:4]
  _month = numbers[0][4:6]
  _day = numbers[0][6:8]

  return _year+'.'+_month+'.'+_day

In [7]:
def make_age(string, keyword):
  arr = []
  arr = string.split(keyword)

  n1 = re.findall('\d+', arr[0])
  n2 = re.findall('\d+', arr[1])

  return str(n1[0])+'('+str(n2[0])+')'

In [8]:
def parse_name(string, keyword):
  arr = []
  arr = string.split(keyword)

  return arr[0]

In [9]:
def parse_sex(string, keyword):
  arr = []
  arr = string.split(keyword)

  return arr[1]

In [10]:
	def download_image(iid,src):
    savename = iid+'.jpg'
    
    urllib.request.urlretrieve(src, IMG_URI+savename)
    
    with Image.open(IMG_URI+savename) as image:
      image.save(IMG_URI+savename, quality=40)

In [11]:
def scrappy(soup):

  seq, kind, name, sex, age, odate, area, wear, src = [], [], [], [], [], [], [], [], []

  contents = soup.select('div div div table tbody tr td dl')

  try:
    tmp = contents[0].select('dt a')[0].get('href')
    tmp = tmp.split("=")[-1]
    seq.append(tmp)
  except ValueError:
    seq.append('seq')

  try:
    tmp = contents[0].select('dt img')[0].get('alt')
    kind.append(tmp)
  except ValueError:
    kind.append('kind')    

  try:
    tmp = contents[0].select('dt a')[0].text
    name.append(parse_name(tmp,'(').strip())
  except ValueError:
    name.append('name')   

  try:
    tmp = contents[0].select('dt a')[0].text
    sex.append(parse_sex(tmp,')').strip())
  except ValueError:
    sex.append('sex')       

  try:
    tmp = contents[0].select('dd')[0].text
    arr = make_array(tmp,':')
    age.append(make_age(arr[1].strip(),'('))
  except ValueError:
    age.append('age')   

  try:
    tmp = contents[0].select('dd')[1].text
    arr = make_array(tmp,':')
    odate.append(make_date(arr[1].strip()))
  except ValueError:
    odate.append('odate')     

  try:
    tmp = contents[0].select('dd')[2].text
    arr = make_array(tmp,':')    
    area.append(arr[1].strip())
  except ValueError:
    area.append('area')    
 
  try:
    tmp = contents[0].select('dd')[3].text
    arr = make_array(tmp,':')
    wear.append(arr[1].strip())
  except ValueError:
    wear.append('wear')   

  try:
    tmp = contents[0].select('dt a')[0].get('href')
    tmp = tmp.split("=")[-1]
    src.append(IMG_URL+tmp+IMG_PRM)
  except ValueError:
    src.append('src')    

  dic_missing= {}

  dic_missing['seq'] = seq
  dic_missing['kind'] = kind
  dic_missing['name'] = name
  dic_missing['sex'] = sex
  dic_missing['age'] = age
  dic_missing['odate'] = odate
  dic_missing['area'] = area
  dic_missing['wear'] = wear
  dic_missing['iid'] = identified()
  dic_missing['src'] = src

  print(dic_missing)

  df_missing = pd.DataFrame(dic_missing)
  print(df_missing)
  df_missing.to_csv('drive/My Drive/development/csv/missing/missing.csv', header=False, index=False, mode='a', encoding='utf-8-sig')

  download_image(identified(),src[0])

# 인증키 발급 주소
http://www.safe182.go.kr/home/api/guideMain.do

# 아래의 깃허브 오픈 소스를 응용하여  pandas CSV 파일 저장 또는 MySQL을 연동하여 데이터를 저장할 수 있습니다.
https://github.com/gyunseul9


In [12]:
url = 'http://www.safe182.go.kr/api/lcm/findChildListT.do'

esntlId = '' #고유아아디 (필수)
authKey = '' #인증키 (필수)
rowSize = 10 #게시물수, 100개까지 (필수)
returnURL = 'http://13.124.230.158' #응답URL (필수)
nowPage = 0 #현재페이지
writngTrgetDscds = '' #대상:정상아동(18세미만)->010,지적장애인->060,치매질환자->070,예:writngTrgetDscds=010
#writngTrgetDscds[] = {'010', '060', '070'};
sexdstnDscd = 0 #성별:남자1,여자2
nm = '' #이름
detailDate1	= '' #시작발생일:2019-01-01
detailDate2 = '' #종료발생일
age1 = 00 #시작당시나이
age2 = 0 #종료당시나이
occrAdres = '' #발생장소

params = {
    'esntlId': esntlId, 
    'authKey': authKey, 
    'rowSize': rowSize, 
    'returnURL': returnURL
}

resp = requests.get(url, params)

with open(HTML_URI+'result.html','w') as file:
  file.write(resp.text)

with open(HTML_URI+'result.html') as fp:
  soup = BeautifulSoup(fp, 'html.parser')
  
  scrappy(soup) 


{'seq': ['4891409'], 'kind': ['아동'], 'name': ['김다은'], 'sex': ['여자'], 'age': ['13(13)'], 'odate': ['2020.08.11'], 'area': ['경상남도 김해시 전하로99번길'], 'wear': ['불상'], 'iid': '202008120011', 'src': ['http://www.safe182.go.kr/blobImgView.do?msspsnIdntfccd=4891409&rptDscd=2']}
       seq  ...                                                src
0  4891409  ...  http://www.safe182.go.kr/blobImgView.do?msspsn...

[1 rows x 10 columns]
